In [281]:
# 1. Συνδέουμε το Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [282]:
import pandas as pd

# Το σωστό path του αρχείου
csv_path = '/content/drive/MyDrive/ML Project/players_21.csv'

# Φορτώνουμε το αρχείο CSV
df = pd.read_csv(csv_path)

# Εμφανίζουμε τις πρώτες 5 γραμμές
df.head()


,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club_name,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,33,1987-06-24,170,72,Argentina,FC Barcelona,...,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,35,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
2,200389,https://sofifa.com/player/200389/jan-oblak/210002,J. Oblak,Jan Oblak,27,1993-01-07,188,87,Slovenia,Atlético Madrid,...,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3
3,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,31,1988-08-21,184,80,Poland,FC Bayern München,...,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,28,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3


In [283]:
# Επιλέγουμε τις στήλες που θέλουμε
selected_columns = ['age', 'height_cm', 'weight_kg', 'short_name', 'nationality', 'club_name', 'overall']
df_selected = df[selected_columns].dropna()

# Εμφάνιση πρώτων γραμμών για έλεγχο
df_selected.head()


,age,height_cm,weight_kg,short_name,nationality,club_name,overall
0,33,170,72,L. Messi,Argentina,FC Barcelona,93
1,35,187,83,Cristiano Ronaldo,Portugal,Juventus,92
2,27,188,87,J. Oblak,Slovenia,Atlético Madrid,91
3,31,184,80,R. Lewandowski,Poland,FC Bayern München,91
4,28,175,68,Neymar Jr,Brazil,Paris Saint-Germain,91


In [284]:
from sklearn.preprocessing import LabelEncoder

# Κάνουμε encoding τα κατηγορικά πεδία
le_nat = LabelEncoder()
le_club = LabelEncoder()

df_selected['nationality'] = le_nat.fit_transform(df_selected['nationality'])
df_selected['club_name'] = le_club.fit_transform(df_selected['club_name'])


In [285]:
from sklearn.model_selection import train_test_split

# Ορίζουμε X (features) και y (στόχος)
X = df_selected.drop('overall', axis=1)
y = df_selected['overall']

# Χώρισμα σε training και testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [286]:
# Φτιάχνουμε σωστά τα X και y χωρίς το short_name
X = df_selected.drop(['overall', 'short_name'], axis=1)
y = df_selected['overall']

# Διαχωρισμός training/testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [289]:
# 📦 Εισαγωγές
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 📂 Φόρτωση αρχείου (π.χ. FIFA21)
df = pd.read_csv('/content/drive/MyDrive/ML Project/players_21.csv')  # άλλαξε το path αν είσαι αλλού

# ✅ Επιλογή απαραίτητων στηλών
columns_to_use = ['overall', 'short_name', 'age', 'height_cm', 'weight_kg', 'nationality', 'club_name']
df_selected = df[columns_to_use].dropna()


# 🔢 Μετατροπή 'overall' σε κατηγορίες
def categorize_rating(overall):
    if overall >= 83:
        return "Top"
    elif overall > 75:
        return "Average"
    else:
        return "Low"

df_selected['rating_category'] = df_selected['overall'].apply(categorize_rating)

# 🧬 Μετατροπή κατηγορικών δεδομένων (Label Encoding)
le_nat = LabelEncoder()
le_club = LabelEncoder()

le_nat.fit(df['nationality'])
le_club.fit(df['club_name'])

df_selected['nationality'] = le_nat.fit_transform(df_selected['nationality'])
df_selected['club_name'] = le_club.fit_transform(df_selected['club_name'])

# 🎯 Ορισμός χαρακτηριστικών (X) και στόχου (y)
X = df_selected.drop(['overall', 'short_name', 'rating_category'], axis=1)
y = df_selected['rating_category']

# ✂️ Διαχωρισμός σε training και test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 🌲 Εκπαίδευση Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
clf.fit(X_train, y_train)

# 📊 Αξιολόγηση μοντέλου
y_pred = clf.predict(X_test)
print("\n✅ Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))



✅ Accuracy: 0.922

📊 Classification Report:
               precision    recall  f1-score   support

     Average       0.33      0.01      0.02       250
         Low       0.92      1.00      0.96      3452
         Top       0.00      0.00      0.00        42

    accuracy                           0.92      3744
   macro avg       0.42      0.34      0.33      3744
weighted avg       0.87      0.92      0.89      3744

